In [12]:
pwd

'/Users/ddang/learning-data-science/learning-data-science/data_analysis/pandas'

In [18]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0   177k      0 --:--:-- --:--:-- --:--:--  178k


In [19]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   3891      0 --:--:-- --:--:-- --:--:--  3901


In [20]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   3145      0 --:--:-- --:--:-- --:--:--  3148


In [21]:
import pandas as pd
import numpy as np

In [24]:
# CSV 파일 읽기
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

# DataFrame의 첫 5행 출력
print(pop.head())
print(areas.head())
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [ ]:
# pop과 abbrevs DataFrame을 'state/region'과 'abbreviation' 열을 기준으로 외부 조인(outer join) 수행하여 병합
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')

# 중복된 'abbreviation' 열 삭제 (axis=1을 명시적으로 지정)
merged = merged.drop('abbreviation', axis=1)  # 중복 정보 삭제

# 병합된 DataFrame의 첫 5행 출력

/var/folders/s4/vwxst1r92574j_j67vlznjq80000gn/T/ipykernel_15866/1415058370.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('abbreviation', 1) # 중복 정보 삭제


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [27]:
# 병합된 DataFrame에서 결측치 확인
print(merged.isnull().any())

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [28]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [29]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

# # 'merged' DataFrame의 열 이름 확인
# print(merged.columns)

# # 'state' 열이 존재하는지 확인하고, 결측치인 'state/region'의 고유값을 출력
# if 'state' in merged.columns:
#     unique_regions = merged.loc[merged['state'].isnull(), 'state/region'].unique()
#     print("결측치인 'state/region'의 고유값:", unique_regions)
# else:
#     print("'state' 열이 존재하지 않습니다. 대신 'state/region' 열을 확인해보세요.")
#     # 'state/region' 열을 기준으로 결측치 확인
#     unique_regions = merged.loc[merged['state/region'].isnull(), 'state/region'].unique()
#     print("결측치인 'state/region'의 고유값:", unique_regions)

array(['PR', 'USA'], dtype=object)

In [30]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [33]:
final = pd.merge(merged, areas, on = 'state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [34]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [35]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [36]:
final.dropna(inplace = True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [37]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [38]:
data2010.set_index('state', inplace = True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending = False, inplace = True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [39]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64